# Análise Exploratória dos Gastos Públicos <br> (Decorrentes da Pandemia de Covid-19 na Paraíba)

Tabelas

- Despesas dos Gastos Publicos <br>
https://drive.google.com/file/d/10I3AfcCdJ2lBQ4_IksjvwG_j6jvxnNR5/view

- Casos de COVID-19 <br>
https://drive.google.com/file/d/1030LEXqE0J24Wf-gES3iNzahLPvVuIGT/view

O intervalo representado nestas tabelas correspondem a 01/03/2020 até 31/12/2021.

## Importações

In [12]:
import calendar
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import textwrap
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [13]:
# Configuração de Variaveis
pd.set_option('display.max_rows', None)

meses_portugues = {
    1: "janeiro",
    2: "fevereiro",
    3: "março",
    4: "abril",
    5: "maio",
    6: "junho",
    7: "julho",
    8: "agosto",
    9: "setembro",
    10: "outubro",
    11: "novembro",
    12: "dezembro"
}

In [14]:
# urls
dataset_despesas_url = 'https://drive.google.com/uc?id=10I3AfcCdJ2lBQ4_IksjvwG_j6jvxnNR5'
dataset_casos_url = 'https://drive.google.com/uc?id=1030LEXqE0J24Wf-gES3iNzahLPvVuIGT'

# Tabela de Despesas
dataset_despesas = pd.read_csv(dataset_despesas_url, parse_dates=['Mês Ano'])

# Tabela de Casos
dataset_casos = pd.read_csv(dataset_casos_url, parse_dates=['date'])

<ipython-input-14-673586f7915b>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dataset_despesas = pd.read_csv(dataset_despesas_url, parse_dates=['Mês Ano'])


## Tratamento de Dados


### Depesas


#### Conversão para Números Flutuantes


In [15]:
def converter_para_float(texto):
    texto = texto.replace('.', '')
    texto = texto.replace(',', '.')

    if '-' in texto:
        return -float(texto.replace('-', ''))
    else:
        return float(texto)

dataset_despesas['Valor Empenhado'] = dataset_despesas['Valor Empenhado'].apply(converter_para_float)
dataset_despesas['Valor Liquidado'] = dataset_despesas['Valor Liquidado'].apply(converter_para_float)
dataset_despesas['Valor Pago'] = dataset_despesas['Valor Pago'].apply(converter_para_float)
dataset_despesas['Valor Restos a pagar Cancelado'] = dataset_despesas['Valor Restos a pagar Cancelado'].apply(converter_para_float)
dataset_despesas['Valor Restos a Pagar Pagos'] = dataset_despesas['Valor Restos a Pagar Pagos'].apply(converter_para_float)

#### Manipulação de datas


In [16]:
dataset_despesas["Mês Ano"]= dataset_despesas["Mês Ano"].str.replace('00/2021', '01/2021')
dataset_despesas["Mês Ano"] = pd.to_datetime(dataset_despesas["Mês Ano"], format='%m/%Y')

dataset_despesas['ano'] = dataset_despesas["Mês Ano"].dt.year
dataset_despesas['mes'] = dataset_despesas["Mês Ano"].dt.month

#### Remoção de Identificação Numérica

In [17]:
dataset_despesas['Elemento de Despesa'] = dataset_despesas['Elemento de Despesa'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Subtítulo (Localizador)'] = dataset_despesas['Subtítulo (Localizador)'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Subfunção'] = dataset_despesas['Subfunção'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Área de atuação (Função)'] = dataset_despesas['Área de atuação (Função)'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Ação Orçamentária'] = dataset_despesas['Ação Orçamentária'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Órgão/Entidade Vinculada'] = dataset_despesas['Órgão/Entidade Vinculada'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Órgão Superior'] = dataset_despesas['Órgão Superior'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Plano Orçamentário'] = dataset_despesas['Plano Orçamentário'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Grupo de Despesa'] = dataset_despesas['Grupo de Despesa'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Elemento de Despesa'] = dataset_despesas['Elemento de Despesa'].str.replace(r'^\d+ - ', '', regex=True)
dataset_despesas['Unidade Gestora'] =  dataset_despesas['Unidade Gestora'].str.replace(r'^\d+ - ', '', regex=True)


#### Filtragem pela Paraíba


 Método utilizado para limitar os dados para PB, usando a coluna Unidade Gestora

In [18]:


dataset_despesas_unidade_gestora = dataset_despesas.loc[dataset_despesas['Unidade Gestora'].str.contains('PARAIBA|PB', case=False)]
dataset_despesas_subtitulo_localizador = dataset_despesas.loc[dataset_despesas['Subtítulo (Localizador)'].str.contains('PARAIBA', case=False)]

dataset_despesas = pd.concat([dataset_despesas_unidade_gestora,dataset_despesas_subtitulo_localizador])

### Casos

#### Limitação de Intervalo
(01/03/2020 - 31/12/2021)

In [19]:
data_inicial_de_casos = pd.to_datetime('2020-03-01')
data_final_de_casos = pd.to_datetime('2021-12-31')

dataset_casos = dataset_casos[(dataset_casos['date'] >= data_inicial_de_casos) & (dataset_casos['date'] <= data_final_de_casos)]

dataset_casos['ano'] = dataset_casos['date'].dt.year
dataset_casos['mes'] = dataset_casos['date'].dt.month

dataset_casos.rename(columns={'date': 'Mês Ano'}, inplace = True)

#### Filtragem pela Paraíba

In [20]:
dataset_casos = dataset_casos[dataset_casos['state'] == 'PB']
dataset_casos['city'] = dataset_casos['city'].str.replace('/PB', '')

#### Tabelas Intermediarias

In [21]:
# Tabela de Casos Por Mês
dataset_casos_por_mes = dataset_casos.groupby(['mes', 'ano'])['newCases'].sum().reset_index()
dataset_casos_por_mes['Mês Ano'] = dataset_casos_por_mes.apply(lambda row: pd.to_datetime(str(row['ano']) + '-' + str(row['mes']) + '-1'), axis=1)
dataset_casos_por_mes.drop(['ano','mes'], axis=1, inplace=True)
dataset_casos_por_mes.rename(columns={'newCases': 'cases'}, inplace = True)

# Tabela de Mortes Por Mês
dataset_mortes_por_mes = dataset_casos.groupby(['mes', 'ano'])['newDeaths'].sum().reset_index()
dataset_mortes_por_mes['Mês Ano'] = dataset_mortes_por_mes.apply(lambda row: pd.to_datetime(str(row['ano']) + '-' + str(row['mes']) + '-1'), axis=1)
dataset_mortes_por_mes.drop(['ano','mes'], axis=1, inplace=True)
dataset_mortes_por_mes.rename(columns={'newDeaths': 'death'}, inplace = True)

## Graficos

### Distribuição de Elementos de Dispensa

In [22]:
dataset_despesas_histograma_elemento_de_dispensa = px.histogram(
    dataset_despesas['Elemento de Despesa'],
    color=dataset_despesas['Elemento de Despesa'],
    title='',
)

dataset_despesas_histograma_elemento_de_dispensa.update_xaxes(showticklabels=False)
dataset_despesas_histograma_elemento_de_dispensa.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend=dict(
        title=dict(text='Elemento de Despesa'),
        x=0.7,
        y=0.95,
        traceorder="normal",
    )
)

dataset_despesas_histograma_elemento_de_dispensa.show()

### Distribuição de Área de Atuação

In [23]:
dataset_despesas_histograma_area_atuacao = px.histogram(
    dataset_despesas['Área de atuação (Função)'],
    color=dataset_despesas['Área de atuação (Função)'],
    title='',
)

dataset_despesas_histograma_area_atuacao.update_xaxes(showticklabels=False)
dataset_despesas_histograma_area_atuacao.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend=dict(
        title=dict(text='Área de Atuação (Função)'),
        x=0.83,
        y=0.95,
        traceorder="normal",
    )
)

dataset_despesas_histograma_area_atuacao.show()

###Distribuição de Sub Função da Área de Atuação

In [24]:
dataset_despesas_histograma_sub_funcao = px.histogram(
    dataset_despesas['Subfunção'],
    color=dataset_despesas['Subfunção'],
    title=''
)

dataset_despesas_histograma_sub_funcao.update_xaxes(showticklabels=False)
dataset_despesas_histograma_sub_funcao.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend=dict(
        title=dict(text='Sub Função'),
        x=0.76,
        y=0.95,
        traceorder="normal",
    )
)

dataset_despesas_histograma_sub_funcao.show()

### Distribuição de Unidade Gestora

In [25]:
dataset_despesas_histograma_unidade = px.histogram(
    dataset_despesas['Unidade Gestora'],
    color=dataset_despesas['Unidade Gestora'],
    title=''
)

dataset_despesas_histograma_unidade.update_xaxes(showticklabels=False)
dataset_despesas_histograma_unidade.update_layout(
    xaxis_title='',
    yaxis_title='',
    legend=dict(
        title=dict(text='Unidade Gestora'),
        x=0.691,
        y=0.991,
        traceorder="normal",
    )
)

dataset_despesas_histograma_unidade.show()

### Variação Mensal de Despesas: Valores Líquidos, Empenhados e Pagos

In [26]:
dataset_despesas_agrupamento_mensal = dataset_despesas.groupby(['ano','mes']).agg({
    'Valor Empenhado': ['mean', 'max'],
    'Valor Liquidado': ['mean', 'max'],
    'Valor Pago': ['mean', 'max']
}).reset_index()

dataset_despesas_agrupamento_mensal['mes'] = dataset_despesas_agrupamento_mensal['mes'].map(lambda x: meses_portugues[x].capitalize())
dataset_despesas_agrupamento_mensal['Mes/Ano'] = dataset_despesas_agrupamento_mensal['mes'].astype(str) + '/' + dataset_despesas_agrupamento_mensal['ano'].astype(str)

dataset_despesas_agrupamento_mensal.drop(['ano', 'mes'], axis=1, inplace=True)

dataset_despesas_agrupamento_mensal = dataset_despesas_agrupamento_mensal[['Mes/Ano', 'Valor Empenhado', 'Valor Liquidado', 'Valor Pago']]

dataset_despesas_agrupamento_mensal_fig = go.Figure()

dataset_despesas_agrupamento_mensal_fig.add_trace(go.Scatter(x=dataset_despesas_agrupamento_mensal['Mes/Ano'], y=dataset_despesas_agrupamento_mensal['Valor Empenhado']['mean'], mode='lines', name='Media dos Valores Empenhados'))
dataset_despesas_agrupamento_mensal_fig.add_trace(go.Scatter(x=dataset_despesas_agrupamento_mensal['Mes/Ano'], y=dataset_despesas_agrupamento_mensal['Valor Liquidado']['mean'], mode='lines', name='Media dos Valores Liquidados'))
dataset_despesas_agrupamento_mensal_fig.add_trace(go.Scatter(x=dataset_despesas_agrupamento_mensal['Mes/Ano'], y=dataset_despesas_agrupamento_mensal['Valor Pago']['mean'], mode='lines', name='Media dos Valores Pagos'))

dataset_despesas_agrupamento_mensal_fig.update_layout(
    title='Variação Mensal de Despesas',
    xaxis_title='Mes/Ano',
    yaxis_title='Valores',
)


<ipython-input-26-210c71febfd8>:10: PerformanceWarning:

dropping on a non-lexsorted multi-index without a level parameter may impact performance.



In [27]:
# Tabela
dataset_despesas_agrupamento_mensal

Mes/Ano Valor Empenhado              Valor Liquidado               \
                              mean          max            mean          max   
0       Abril/2020    7.462559e+04    548493.86      829.016000      3942.06   
1        Maio/2020    1.471568e+06  10900000.00   859693.189474  10800000.00   
2       Junho/2020    5.054426e+04    767329.78   701405.901250   8799996.00   
3       Julho/2020    1.212900e+05   2183417.51    40910.290000    767626.81   
4      Agosto/2020    7.524458e+05  10477577.11   167682.534286   1679292.96   
5    Setembro/2020   -1.948289e+03       284.95   157211.071333   2261809.18   
6     Outubro/2020   -1.439094e+03      5618.43   125880.998667   1635604.76   
7    Novembro/2020    7.569965e+04    629247.39    89370.950000    991592.24   
8    Dezembro/2020    1.705025e+04    279917.43   315744.074783   6092398.45   
9     Janeiro/2021    0.000000e+00         0.00        0.000000         0.00   
10  Fevereiro/2021    1.080818e+03     10089.00        0.000000         0.00   
11      Março/2021    0.000000e+00         0.00     1565.508000      7827.54   
12      Abril/2021    0.000000e+00         0.00      253.360000       760.08   
13       Maio/2021   -1.253450e+02      1000.00      450.000000      1800.00   
14      Junho/2021    0.000000e+00         0.00      333.333333      1000.00   
15      Julho/2021    2.128350e+03      2128.35        0.000000         0.00   
16     Agosto/2021    0.000000e+00         0.00     1064.175000      2128.35   
17   Setembro/2021    3.990496e+04    108999.75        0.000000         0.00   
18    Outubro/2021    5.872500e+03     17617.50        0.000000         0.00   
19   Novembro/2021    8.345863e+04    442834.28      726.700000      2906.80   
20   Dezembro/2021    3.194293e+05   3686109.28    63387.733077    442834.28   

       Valor Pago               
             mean          max  
0      580.056000      3942.06  
1   853003.341053  10800000.00  
2   707297.463750   8799996.00  
3    32688.984091    557094.66  
4   162716.693571   1766383.69  
5    93020.173333   1223743.04  
6   160243.620000   2080285.17  
7    88190.436471   1073438.31  
8   296867.831739   5791522.72  
9        0.000000         0.00  
10       0.000000         0.00  
11    1565.508000      7827.54  
12     253.360000       760.08  
13     450.000000      1800.00  
14       0.000000         0.00  
15       0.000000         0.00  
16     710.000000      1000.00  
17     244.050000      1708.35  
18       0.000000         0.00  
19     726.700000      2906.80  
20   63387.733077    442834.28

In [28]:
# Grafico
dataset_despesas_agrupamento_mensal_fig.show()

### Correlação entre Valor Pago e Área de Atuação

In [29]:
dataset_despesas_correlacao_valor_pago_e_area_de_atuacao = dataset_despesas.groupby(["Área de atuação (Função)"])['Valor Pago'].sum().reset_index()

dataset_despesas_correlacao_valor_pago_e_area_de_atuacao_fig = px.bar(
    dataset_despesas_correlacao_valor_pago_e_area_de_atuacao,
    x='Área de atuação (Função)',
    y='Valor Pago',
    color='Área de atuação (Função)',
    text_auto='.2s',
    labels=dict(x='', y=''),
    title='',
)

dataset_despesas_correlacao_valor_pago_e_area_de_atuacao_fig.update_yaxes(tickformat=',.2f')
dataset_despesas_correlacao_valor_pago_e_area_de_atuacao_fig.update_layout(
    xaxis=dict(title=''),
    yaxis=dict(title=''),
    yaxis_tickprefix='R$ ',
    legend=dict(
        x=0.8,
        y=0.95,
        traceorder="normal",
    ),
     font=dict(size=18)
)

dataset_despesas_correlacao_valor_pago_e_area_de_atuacao_fig.show()

### Correlação entre Valor Pago e Elemento de Despesa

In [30]:
dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa = dataset_despesas.groupby(["Elemento de Despesa"])['Valor Pago'].sum().reset_index()

dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa_fig = px.bar(
    dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa,
    x='Elemento de Despesa',
    y='Valor Pago',
    color='Elemento de Despesa',
    text_auto='.2s',
    labels=dict(x='', y=''),
    title='',
)

dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa_fig.update_yaxes(tickformat=',.2f')
dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa_fig.update_xaxes(showticklabels=False)
dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa_fig.update_layout(
    yaxis=dict(title=''),
    yaxis_tickprefix='R$ ',
    legend=dict(
        x=0.69,
        y=0.95,
        traceorder="normal",
    )
)

dataset_despesas_correlacao_valor_pago_e_elemento_de_despesa_fig.show()

### Correlação entre Valor Pago e Área de Atuação (Função)

In [31]:
dataset_despesas_correlacao_valor_pago_e_funcao = dataset_despesas.groupby(["Área de atuação (Função)"])['Valor Pago'].sum().reset_index()

dataset_despesas_correlacao_valor_pago_e_funcao_fig = px.bar(
    dataset_despesas_correlacao_valor_pago_e_funcao,
    x='Área de atuação (Função)',
    y='Valor Pago',
    color='Área de atuação (Função)',
    text_auto='.2s',
    labels=dict(x='', y=''),
    title='',
)

dataset_despesas_correlacao_valor_pago_e_funcao_fig.update_yaxes(tickformat=',.2f')
dataset_despesas_correlacao_valor_pago_e_funcao_fig.update_xaxes(showticklabels=False)
dataset_despesas_correlacao_valor_pago_e_funcao_fig.update_layout(
    yaxis=dict(title=''),
    yaxis_tickprefix='R$ ',
    legend=dict(
        x=0.79,
        y=0.95,
        traceorder="normal",
    )
)

dataset_despesas_correlacao_valor_pago_e_funcao_fig.show()

### Comparação do Valor Empenhado e a Quantidade de Casos

In [32]:
scaler = MinMaxScaler()

merge_casos_por_mes_despesas = pd.merge(dataset_despesas, dataset_casos_por_mes, on='Mês Ano')
merge_casos_por_mes_despesas_group =  merge_casos_por_mes_despesas.groupby(['Mês Ano'])[['Valor Empenhado','Valor Pago','cases']].max().reset_index()
merge_casos_por_mes_despesas_group['date'] = merge_casos_por_mes_despesas_group['Mês Ano'].dt.to_period('M').astype(str)



merge_casos_por_mes_despesas_group['valor_empenhado_norm'] = scaler.fit_transform(merge_casos_por_mes_despesas_group[['Valor Empenhado']])
merge_casos_por_mes_despesas_group['cases_norm'] = scaler.fit_transform(merge_casos_por_mes_despesas_group[['cases']])


merge_casos_por_mes_despesas_group_fig = go.Figure()

merge_casos_por_mes_despesas_group_fig.add_trace(go.Scatter(x=merge_casos_por_mes_despesas_group['date'], y=merge_casos_por_mes_despesas_group['valor_empenhado_norm'], mode='lines', name='Valor Empenhado'))
merge_casos_por_mes_despesas_group_fig.add_trace(go.Scatter(x=merge_casos_por_mes_despesas_group['date'], y=merge_casos_por_mes_despesas_group['cases_norm'], mode='lines', name='Casos'))

merge_casos_por_mes_despesas_group_fig.update_layout(
    title='Comparação do Valor Empenhado com a Quantidade de Casos',
    xaxis_title='Mes/Ano',
    yaxis_title='Valores'
)

merge_casos_por_mes_despesas_group_fig.show()

### Evolução Mensal dos Casos e Mortes

In [33]:
dataset_casos_evolucao = pd.DataFrame()

dataset_casos_evolucao['Casos'] = dataset_casos.groupby(['ano','mes'])['newCases'].sum()
dataset_casos_evolucao['Mortes'] = dataset_casos.groupby(['ano','mes'])['newDeaths'].sum()

dataset_casos_evolucao = dataset_casos_evolucao.reset_index()

dataset_casos_evolucao['mes'] = dataset_casos_evolucao['mes'].map(lambda x:meses_portugues[x].capitalize())
dataset_casos_evolucao['Mes/Ano'] = dataset_casos_evolucao['mes'].astype(str) + '/' + dataset_casos_evolucao['ano'].astype(str)

dataset_casos_evolucao.drop(['ano', 'mes'], axis=1, inplace=True)

colunas_no_dataset_casos_evolucao = ['Mes/Ano'] + [col for col in dataset_casos_evolucao.columns if col != 'Mes/Ano']
dataset_casos_evolucao = dataset_casos_evolucao[colunas_no_dataset_casos_evolucao]

dataset_casos_evolucao_fig = go.Figure()

dataset_casos_evolucao_fig.add_trace(go.Scatter(x=dataset_casos_evolucao['Mes/Ano'], y=dataset_casos_evolucao['Casos'], mode='lines', name='Total de Casos'))
dataset_casos_evolucao_fig.add_trace(go.Scatter(x=dataset_casos_evolucao['Mes/Ano'], y=dataset_casos_evolucao['Mortes'], mode='lines', name='Total das Mortes'))

dataset_casos_evolucao_fig.update_layout(
    title='Evolução Mensal dos Casos e Mortes',
    xaxis_title='Mes/Ano',
    yaxis_title='Valores',
    legend=dict(
      orientation="h",
      yanchor="bottom",
      y=1.02,
      xanchor="right",
      x=1
  ),
)

dataset_casos_evolucao_fig.update_traces(textposition="top right")


In [34]:
# Tabela
dataset_casos_evolucao

,Mes/Ano,Casos,Mortes
0,Março/2020,18,1
1,Abril/2020,908,66
2,Maio/2020,12237,294
3,Junho/2020,33800,617
4,Julho/2020,35832,834
5,Agosto/2020,22984,639
6,Setembro/2020,15489,374
7,Outubro/2020,11882,277
8,Novembro/2020,12104,194
9,Dezembro/2020,21231,377


In [35]:
# Grafico
dataset_casos_evolucao_fig.show()